In [1]:
import os
import requests


In [2]:
model = "VertexGemini"
coreID = "<CORE_ID>"
API_KEY = "<KEY>"

In [3]:
# Define the API details
host = "https://genai-service.stage.commandcentral.com/app-gateway"
endpoint = "/api/v2/chat"
url = f"{host}{endpoint}"

headers = {
    "x-msi-genai-api-key": API_KEY,
    "Content-Type": "application/json",
}

## Commands and original prompt

In [4]:
DRONE_API_COMMANDS = [
    "takeoff()",
    "land()",
    "move_forward(distance_cm)",
    "move_backward(distance_cm)",
    "move_left(distance_cm)",
    "move_right(distance_cm)",
    "move_up(distance_cm)",
    "move_down(distance_cm)",
    "turn_left(degrees)",
    "turn_right(degrees)",
    "flip()",
    "go_to_xyz(x_cm, y_cm, z_cm)", # Example: absolute position
    "set_speed(speed_percent)",
    "hover(duration_seconds)",
    "start_video_record()",
    "stop_video_record()",
    "take_photo()",
    "emergency_stop()",
    "get_battery_status()",
    "get_current_position()",
    # Add your remaining specific drone commands here
]

In [5]:
start_message_content = (
        "You are an intelligent drone command translator. Your sole purpose is to "
        "convert natural language sentences into precise drone API commands. "
        "Strictly adhere to the following rules:\n"
        "1. All distance measurements (e.g., meters, feet, inches) must be converted to centimeters (cm).\n"
        "2. Angles must remain in degrees.\n"
        "3. Your output must ONLY be the API command, with no extra text, explanations, or formatting.\n"
        "4. Use only the following available API commands and their specified parameter types:\n" +
        "\n".join([f"- {cmd}" for cmd in DRONE_API_COMMANDS]) +
        "\n\nExample 1:\nInput: Move forward by 2 meters.\nOutput: move_forward(200)\n"
        "Example 2:\nInput: Go up 0.5 meters.\nOutput: move_up(50)\n"
        "Example 3:\nInput: Turn left 45 degrees.\nOutput: turn_left(45)\n"
        "Example 4:\nInput: Flip the drone.\nOutput: flip()\n"
        "Example 5:\nInput: Move right for 1 foot.\nOutput: move_right(30.48)\n" # 1 foot = 30.48 cm
        "Example 6:\nInput: Land.\nOutput: land()\n"
        "Example 7:\nInput: start recording.\nOutput: start_video_record()\n"
        "Example 8:\nInput: move to 1 meter forward, 0.5 meters right, and 20 cm up from origin.\nOutput: go_to_xyz(100, 50, 20)\n"
        "If a command cannot be clearly mapped to one of the provided API commands, output 'UNKNOWN_COMMAND'.\n"
)

## Functions

In [6]:
payload = {
    "userId": coreID + "@motorolasolutions.com",
    "model": model,
    "prompt": start_message_content,
}

# Make the POST request
response = requests.post(url, headers=headers, json=payload)
data = response.json()
# Check the response
if response.status_code != 200:
    print("Error:", response.status_code, response.text)
print(response.json()['msg'])
session_id = data.get("sessionId")


```



In [7]:
# --- Function to convert user input to API command using GPT ---

def get_drone_api_command(user_sentence: str, session_id: str) -> str:
    """
    Takes a natural language sentence, uses a GPT model to convert it into a
    drone API command, ensuring all measurements are in centimeters.

    Args:
        user_sentence (str): The natural language description of the drone task.

    Returns:
        str: The corresponding API command (e.g., "move_forward(100)") or an error message.
    """

    payload = {
        "userId": coreID + "@motorolasolutions.com",
        "model": model,
        "prompt": user_sentence,
        "session_id": session_id,
        "system": start_message_content
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=payload)

    # Check the response
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)

    return response.json()['msg']

## User input

In [8]:
# Get input from the user
# user_task = input("Enter the drone task sentence: ")
user_task = "move forward 10 feet"

generated_command = get_drone_api_command(user_task, session_id)

print(generated_command)


move_forward(304.8)

